# Evaluación de Modelos V3 con GroupKFold por Entidad

**Objetivo del notebook:**  
Evaluar el rendimiento de distintos modelos de clasificación (versión 3 de features) utilizando validación cruzada basada en grupos (`GroupKFold`), donde cada grupo corresponde a una `entidad`. El objetivo es simular un entorno generalista, asegurando que los datos de cada entidad no se mezclan entre folds.

**Reglas del experimento:**  
- **No recalcular features:** El notebook utiliza el dataset preprocesado en `data/interim/dataset_v3_features.csv` (features V3).
- **No modificar ni utilizar features o scripts de la versión v2.**
- **Validación cruzada por grupos:** Se utiliza `GroupKFold` agrupando por la columna `entidad`.
- **Métrica principal:** El criterio de comparación será la métrica F1.

**Outputs esperados:**  
Al finalizar, se generará una tabla resumen con el rendimiento (media y desviación estándar) de cada modelo evaluado según la métrica principal.


In [13]:
# %%
import numpy as np
import pandas as pd

from sklearn.model_selection import GroupKFold
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# Constantes
DATASET_PATH = "../data/interim/dataset_v3_features.csv"
RANDOM_STATE = 42
FEATURES_V3 = [
    "domain_complexity",
    "domain_whitelist",
    "trusted_token_context",
    "host_entropy",
    "infra_risk",
    "brand_in_path",
    "brand_match_flag"
]

TARGET = "label"
GROUP_KEY = "entidad"



In [14]:
# %%
df = pd.read_csv(DATASET_PATH)

print("Shape:", df.shape)
print("Columnas:", list(df.columns))

print("\nEntidades únicas:", df[GROUP_KEY].nunique())
print("\nDistribución de la variable target:")
print(df[TARGET].value_counts(normalize=True))



Shape: (482, 13)
Columnas: ['url', 'label', 'sector', 'entidad', 'notas', 'campaign', 'domain_complexity', 'domain_whitelist', 'trusted_token_context', 'host_entropy', 'infra_risk', 'brand_in_path', 'brand_match_flag']

Entidades únicas: 110

Distribución de la variable target:
label
0    0.5
1    0.5
Name: proportion, dtype: float64


In [15]:
# %%
# 1. Verificar que todas las columnas en FEATURES_V3 existen en df
for col in FEATURES_V3:
    assert col in df.columns, f"Falta la columna de feature contractual: {col}"

# 2. Verificar que la columna TARGET existe en df
assert TARGET in df.columns, f"Falta la columna objetivo: {TARGET}"

# 3. Verificar que la columna GROUP_KEY existe en df y no tiene nulos
assert GROUP_KEY in df.columns, f"Falta la columna de grupo: {GROUP_KEY}"
assert df[GROUP_KEY].isnull().sum() == 0, "Existen valores nulos en la columna de grupo (entidad)"

# 4. Verificar que df[TARGET] solo contiene {0, 1}
labels_unicos = set(df[TARGET].unique())
assert labels_unicos <= {0, 1}, f"La variable target contiene valores no binarios: {labels_unicos}"

print("Contract checks passed.")



Contract checks passed.


In [16]:
# %%
counts = df[GROUP_KEY].value_counts()

print("Descripción del número de muestras por entidad:")
print(counts.describe())

n_ent_1 = (counts == 1).sum()
n_ent_2 = (counts == 2).sum()
n_ent_3mas = (counts >= 3).sum()

print(f"\nEntidades con solo 1 muestra: {n_ent_1}")
print(f"Entidades con 2 muestras: {n_ent_2}")
print(f"Entidades con 3 o más muestras: {n_ent_3mas}")

print("\nSanity check por entidad completado.")



Descripción del número de muestras por entidad:
count    110.000000
mean       4.381818
std       10.063999
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max       73.000000
Name: count, dtype: float64

Entidades con solo 1 muestra: 50
Entidades con 2 muestras: 30
Entidades con 3 o más muestras: 30

Sanity check por entidad completado.


In [17]:
# %%
gkf = GroupKFold(n_splits=5)

for i, (_, test_idx) in enumerate(gkf.split(df, df[TARGET], groups=df[GROUP_KEY])):
    entidades_test = df.iloc[test_idx][GROUP_KEY].nunique()
    muestras_test = len(test_idx)
    print(f"Fold {i+1}: {entidades_test} entidades en test, {muestras_test} muestras en test")

print("GroupKFold (k=5) configurado correctamente.")



Fold 1: 17 entidades en test, 97 muestras en test
Fold 2: 22 entidades en test, 97 muestras en test
Fold 3: 23 entidades en test, 96 muestras en test
Fold 4: 23 entidades en test, 96 muestras en test
Fold 5: 25 entidades en test, 96 muestras en test
GroupKFold (k=5) configurado correctamente.


In [18]:
# %%
models = {
    "logreg": LogisticRegression(
        solver="lbfgs",
        max_iter=1000,
        class_weight="balanced",
        random_state=RANDOM_STATE
    ),
    "rf": RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_split=2,
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
}

print("Modelos definidos: ", list(models.keys()))



Modelos definidos:  ['logreg', 'rf']


In [20]:
# %%
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

svc_clf = CalibratedClassifierCV(
    estimator=LinearSVC(C=1.0, random_state=RANDOM_STATE),
    method="sigmoid",
    cv=5
)

models["linearsvc"] = svc_clf

print("Modelo añadido:", list(models.keys()))


Modelo añadido: ['logreg', 'rf', 'linearsvc']


In [21]:
# %%
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from scipy.special import expit as sigmoid

results = {model_name: [] for model_name in models.keys()}

for fold_idx, (train_idx, test_idx) in enumerate(gkf.split(df, df[TARGET], groups=df[GROUP_KEY])):
    X_train = df.iloc[train_idx][FEATURES_V3]
    y_train = df.iloc[train_idx][TARGET]
    X_test = df.iloc[test_idx][FEATURES_V3]
    y_test = df.iloc[test_idx][TARGET]
    
    for model_name, model in models.items():
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        if hasattr(clf, "predict_proba"):
            y_proba = clf.predict_proba(X_test)[:,1]
        else:
            y_scores = clf.decision_function(X_test)
            y_proba = sigmoid(y_scores)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        # Manejo robusto de roc_auc (evita error si sólo hay una clase en test)
        try:
            roc = roc_auc_score(y_test, y_proba)
        except Exception:
            roc = float('nan')
        results[model_name].append({
            'fold': fold_idx+1,
            'precision': prec,
            'recall': rec,
            'f1': f1,
            'roc_auc': roc
        })

print("Cross-validation completada.")



Cross-validation completada.


In [22]:
# %%
rows = []
for model_name, folds in results.items():
    precs = [fold['precision'] for fold in folds]
    recs = [fold['recall'] for fold in folds]
    f1s = [fold['f1'] for fold in folds]
    rocs = [fold['roc_auc'] for fold in folds]
    rows.append({
        "model": model_name,
        "precision_mean": np.mean(precs),
        "precision_std": np.std(precs),
        "recall_mean": np.mean(recs),
        "recall_std": np.std(recs),
        "f1_mean": np.mean(f1s),
        "f1_std": np.std(f1s),
        "roc_auc_mean": np.nanmean(rocs),  # ignora NaN
        "roc_auc_std": np.nanstd(rocs)     # ignora NaN
    })

summary_df = pd.DataFrame(rows)[[
    "model",
    "precision_mean", "precision_std",
    "recall_mean", "recall_std",
    "f1_mean", "f1_std",
    "roc_auc_mean", "roc_auc_std"
]]
summary_df = summary_df.sort_values(by="f1_mean", ascending=False).reset_index(drop=True)

print(summary_df)
print("Resumen de métricas generado.")



       model  precision_mean  precision_std  recall_mean  recall_std  \
0  linearsvc        0.893825       0.072115     0.921723    0.053646   
1         rf        0.916678       0.030643     0.891112    0.089996   
2     logreg        0.883610       0.072299     0.926786    0.064648   

    f1_mean    f1_std  roc_auc_mean  roc_auc_std  
0  0.906085  0.054124      0.966365     0.022264  
1  0.902496  0.061998      0.957696     0.024732  
2  0.902171  0.052126      0.963844     0.024387  
Resumen de métricas generado.


In [23]:
# %%
df_sorted = summary_df.sort_values(by="f1_mean", ascending=False).reset_index(drop=True)

if len(df_sorted) > 1 and (df_sorted.iloc[0]["f1_mean"] - df_sorted.iloc[1]["f1_mean"]) < 0.002:
    top = df_sorted[df_sorted["f1_mean"] == df_sorted.iloc[0]["f1_mean"]]
    if len(top) > 1:
        top = top.sort_values(by="roc_auc_mean", ascending=False).reset_index(drop=True)
        if len(top) > 1 and top.iloc[0]["roc_auc_mean"] == top.iloc[1]["roc_auc_mean"]:
            top = top.sort_values(by="recall_mean", ascending=False).reset_index(drop=True)
    best_model_name = top.iloc[0]["model"]
else:
    best_model_name = df_sorted.iloc[0]["model"]
best_model = models[best_model_name]

print("Modelo ganador:", best_model_name)
print(summary_df.loc[summary_df["model"] == best_model_name])



Modelo ganador: linearsvc
       model  precision_mean  precision_std  recall_mean  recall_std  \
0  linearsvc        0.893825       0.072115     0.921723    0.053646   

    f1_mean    f1_std  roc_auc_mean  roc_auc_std  
0  0.906085  0.054124      0.966365     0.022264  


In [26]:
# %%
# Entrenamiento EXPLORATORIO del modelo ganador (NO contractual)
# Solo para análisis posterior (curvas, probas, threshold tuning)

X = df[FEATURES_V3]
y = df[TARGET]

best_model.fit(X, y)

print("Modelo re-entrenado para análisis (no exportado).")


Modelo re-entrenado para análisis (no exportado).


# Conclusiones de la evaluación por entidad (GroupKFold)

Tras la evaluación con validación cruzada por entidad, el modelo `linearsvc` ha sido seleccionado como el ganador. La decisión se basa en que obtiene el mejor valor de F1_mean y de ROC-AUC_mean, mostrando además un recall competitivo y un buen equilibrio entre todas las métricas principales. Considera también la menor varianza relativa de sus métricas frente a Random Forest (RF), lo que aporta una mayor robustez en escenarios con entidades disjuntas.

Desde una perspectiva técnica, la estabilidad de las métricas a nivel de entidad es crucial en detección de phishing enfocado a España, ya que permite una generalización más efectiva y minimiza el riesgo de sobreajuste a entidades individuales, mejorando la detección en entidades desconocidas o futuras.

Por todo ello, consideramos este modelo como baseline v3 listo para la fase de exportación y posterior integración.
